In [1]:
import matplotlib.pyplot as plt
% load_ext autoreload
% autoreload 2
% matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore')

import torch
import torch.nn as nn
from torchvision import transforms
import sys
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download('punkt')


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 3        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 2048           # dimensionality of image and word embeddings
hidden_size = 2048          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\steve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.20s)
creating index...
index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 591753/591753 [00:58<00:00, 10106.22it/s]


In [3]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

losses = []

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))
        
        target_captions = captions[:, 1:].to(device)
        training_captions = captions[:, :captions.shape[1]-1].to(device)
        
        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
#         captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, training_captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), target_captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        losses.append(loss.item())
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        print("\nSaving the model")
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/2], Step [100/9247], Loss: 4.9134, Perplexity: 136.0949
Epoch [1/2], Step [200/9247], Loss: 4.9119, Perplexity: 135.8963
Epoch [1/2], Step [300/9247], Loss: 5.1109, Perplexity: 165.8149
Epoch [1/2], Step [400/9247], Loss: 4.8898, Perplexity: 132.9264
Epoch [1/2], Step [500/9247], Loss: 4.8505, Perplexity: 127.7997
Epoch [1/2], Step [600/9247], Loss: 4.9295, Perplexity: 138.3126
Epoch [1/2], Step [700/9247], Loss: 4.7062, Perplexity: 110.6267
Epoch [1/2], Step [800/9247], Loss: 4.9173, Perplexity: 136.6268
Epoch [1/2], Step [900/9247], Loss: 4.6616, Perplexity: 105.8053
Epoch [1/2], Step [1000/9247], Loss: 5.3972, Perplexity: 220.7977
Epoch [1/2], Step [1100/9247], Loss: 5.0442, Perplexity: 155.1211
Epoch [1/2], Step [1200/9247], Loss: 4.8480, Perplexity: 127.4914
Epoch [1/2], Step [1300/9247], Loss: 5.1895, Perplexity: 179.3877
Epoch [1/2], Step [1400/9247], Loss: 4.8009, Perplexity: 121.6219
Epoch [1/2], Step [1500/9247], Loss: 5.1392, Perplexity: 170.5714
Epoch [1/2], Step [

In [ ]:
plt.plot(losses)